## Import and preprocess datasets

In [3]:
import numpy as np
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from sklearn.ensemble.forest import RandomForestRegressor, RandomForestClassifier
import os
from sklearn import preprocessing, metrics
from importlib import reload
import sknn.mlp
from sklearn import tree  #for graphing random forest tree
import pickle
%matplotlib inline
pylab.rcParams['figure.figsize'] = (10, 6)
inline_rc = dict(mpl.rcParams)
import src.nnload as nnload
import src.nntrain as nntrain
import src.nnnetcdf as nnnetcdf
import src.nnplot as nnplot

# Set script parameters
minlev = 0.0
rainonly = False
evaluate_classifiers = False
evaluate_regressors = False
write_nn_to_netcdf = False
fig_dir = '/Users/jgdwyer/mitbox/scripts/nn/figs/'
data_dir = '/Users/jgdwyer/mitbox/scripts/nn/data/'

# Load data
x, y, Pout, lat, lev, dlev, timestep = nnload.loaddata(data_dir + 'nntest.nc', minlev)

# Ensure that input and outputs are lined up in time!
x=x[1:,:]
y=y[0:-1,:]

# Print some statistics about rain and limit to when it's raining if True
x, y, Pout = nnload.limitrain(x, y, Pout, rainonly)
    
# Store when convection occurs
cv = nnload.whenconvection(y)
        
# Normalize input using the scikit-learn preprocessing tools
# Constrain inputs and outputs between 0 and 1
# Since outputs are sparse, don't shift the mean
scaler_x = preprocessing.MinMaxScaler(feature_range=(-1.0,1.0))
scaler_y = preprocessing.MaxAbsScaler()

# Randomly choose samples
samples = np.random.choice(x.shape[0], x.shape[0], replace=False)
x1,x2,x3    = nnload.pp(x,  samples, scaler_x)
y1,y2,y3    = nnload.pp(y,  samples, scaler_y)
cv1,cv2,cv3 = nnload.pp(cv, samples, None, scale_data=False)

There is some amount of rain 61.0% of the time
There is a rate of >3 mm/day 20.2% of the time
There is convection 65.5% of the time


## Create, train, evaluate, and store regressors NN's

In [ ]:
# Build, train, and save model
from importlib import reload
reload(nntrain)
n_iter = 10000
batch_size = 100
r_mlp, r_str                  = nntrain.build_nn('regress',['Rectifier'],[600],batch_size,'momentum',
                                                 n_stable=100, regularize = 'L2', weight_decay = 0.0001)
print(r_str)
r_mlp, r_errors, r_besterrors = nntrain.train_nn(r_mlp,r_str,x1,y1,x2,y2)
pickle.dump([r_mlp, r_str, r_errors, r_besterrors],  open(data_dir + 'regressors/' + r_str + '.pkl', 'wb'))

# Save model in netcdf format if requested
if write_nn_to_netcdf:
    nnnetcdf.write_netcdf_twolayer(r_mlp, 'regress', datadir + 'netcdf_models/' + r_str + '_neural_weights_regressor.nc')

r_600R_mom0.9reg0.0001


## Compare all NNs to each other

In [ ]:
# Initialize regressors
trained_regressors = ['r_15R_mom0.9','r_16R_mom0.9','r_17R_mom0.9','r_800R_mom0.9','r_800R_mom0.9reg0.0001']
N_r = len(trained_regressors)
r_mlp = [0]*N_r
r_str = [0]*N_r
r_errors=np.empty((n_iter,N_r))
r_besterrors=np.empty((n_iter,N_r))
r_errors[:]=np.NAN
r_besterrors[:]=np.NAN

# Load regressors
for i,r in enumerate(trained_regressors):
    r_mlp[i], r_str[i], err, berr = pickle.load(open(data_dir + 'regressors/' + r + '.pkl', 'rb'))
    r_errors[:len(err),i]      =  err
    r_besterrors[:len(berr),i] = berr
# Compare them and plot output
nntrain.plot_regressors_scores(r_mlp,r_str,x2,y2,fig_dir + 'compare/','r')
nntrain.plot_model_error_over_time(r_errors, r_besterrors, r_str, fig_dir + 'compare/','r')

## Plot test data evaluations

In [ ]:
# Load the model we want to do a detailed evaulation for
r_str_eval = 'r_800R_mom0.9reg0.0001'
r_mlp_eval, _, _, _ = pickle.load(open(data_dir + 'regressors/' + r_str_eval + '.pkl', 'rb'))

# Set figure path and create directory if it does not exist
figpath = fig_dir + r_str_eval + '/'
if not os.path.exists(figpath):
    os.makedirs(figpath)
    
# Set the true and predicted test data
y3_true = y2
y3_pred = r_mlp_eval.predict(x2)

# Plot means and standard deviations
nnplot.plot_means_stds(y3_true, y3_pred, lev, figpath)

# Plot correlation coefficient, explained variance, and rmse
nnplot.plot_error_stats(y3_true, y3_pred, lev, figpath)

# Plot a time series of precipitaiton
nnplot.plot_precip(y3_true, y3_pred, dlev, figpath)

# Plot the enthalpy conservation
nnplot.plot_enthalpy(y3_true, y3_pred, dlev, figpath)

In [ ]:
#     r_mlp.append(build_nn('regress',['Rectifier','Rectifier']            ,[500,500]    ,n_iter,batch_size,'momentum'))
#     r_mlp.append(build_nn('regress',['Rectifier','Rectifier','Rectifier'],[200,200,200],n_iter,batch_size,'momentum'))
#     r_mlp.append(build_nn('regress',['Tanh','Tanh','Tanh']               ,[100,100,100],n_iter,batch_size,'momentum'))
#     r_mlp.append(build_nn('regress',['Tanh','Tanh']                      ,[500,500]    ,n_iter,batch_size,'momentum'))
#     r_mlp.append(build_nn('regress',['Tanh','Tanh']                      ,[200,200]    ,n_iter,batch_size,'momentum'))
#     r_mlp.append(build_nn('regress',['Tanh']                             ,[500]        ,n_iter,batch_size,'momentum'))
#     r_mlp.append(build_nn('regress',['Tanh','Tanh']                      ,[500,500]    ,n_iter,batch_size,'momentum',learning_momentum=0.7))
#     r_mlp.append(build_nn('regress',['Tanh','Tanh']                      ,[500,500]    ,n_iter,batch_size,'sgd'))

In [4]:
import pickle
r_str_eval = 'r_800R_mom0.9reg0.0001'
r_mlp_eval, _, _, _ = pickle.load(open(data_dir + 'regressors/' + r_str_eval + '.pkl', 'rb'))



In [5]:
r_mlp_eval

Regressor(batch_size=100,
     callback={'on_epoch_finish': <function store_stats at 0x114136048>},
     debug=False, dropout_rate=None, f_stable=0.001,
     hidden0=<sknn.nn.Layer `Rectifier`: name='hidden0', units=800, frozen=False>,
     layers=[<sknn.nn.Layer `Rectifier`: name='hidden0', units=800, frozen=False>, <sknn.nn.Layer `Linear`: name='output', units=60, frozen=False>],
     learning_momentum=0.9, learning_rate=0.01, learning_rule='momentum',
     loss_type=None, n_iter=10000, n_stable=10000, normalize=None,
     output=<sknn.nn.Layer `Linear`: name='output', units=60, frozen=False>,
     parameters=None, random_state=None, regularize='L2', valid_set=None,
     valid_size=0.0, verbose=None, warning=None, weight_decay=0.0001)